https://dacon.io/codeshare/4245

In [1]:
## 라이브러리 불러오기
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import glob
import torch

In [2]:
train_mask_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/train/mask/*.png')
test_mask_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/test/mask/*.png')
val_mask_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/val/mask/*.png')
train_original_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/train/original/*.png')
test_original_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/test/original/*.png')
val_original_paths = glob.glob('/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/val/original/*.png')

train_mask_list = []
test_mask_list = []
val_mask_list = []
train_original_list = []
test_original_list = []
val_original_list = []

dir_save_train_np = 'C:/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/train_np'
dir_save_val_np = 'C:/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/val_np'
dir_save_test_np = 'C:/Users/KDJ/Desktop/KDJ/Code/data/BCCD_Dataset_with_mask/test_np'

# if not os.path.exists(dir_save_train_np):
#     os.makedirs(dir_save_train_np)

# if not os.path.exists(dir_save_val_np):
#     os.makedirs(dir_save_val_np)

# if not os.path.exists(dir_save_test_np):
#     os.makedirs(dir_save_test_np)

# for train_mask_path, test_mask_path, val_mask_path, train_original_path, test_original_path, val_original_path in zip(train_mask_paths, test_mask_paths, val_mask_paths, train_original_paths, test_original_paths, val_original_paths):
#     train_mask_list.append(np.array(Image.open(train_mask_path).resize((256, 256)).convert('L')))
#     test_mask_list.append(np.array(Image.open(test_mask_path).resize((256, 256)).convert('L')))
#     val_mask_list.append(np.array(Image.open(val_mask_path).resize((256, 256)).convert('L')))
#     train_original_list.append(np.array(Image.open(train_original_path).resize((256, 256)).convert('L')))
#     test_original_list.append(np.array(Image.open(test_original_path).resize((256, 256)).convert('L')))
#     val_original_list.append(np.array(Image.open(val_original_path).resize((256, 256)).convert('L')))

# ## 선택된 train 이미지를 npy 파일로 저장
# for i, (train_mask, train_original) in enumerate(zip(train_mask_list, train_original_list)):

#     label_ = np.asarray(train_mask)
#     input_ = np.asarray(train_original)

#     np.save(os.path.join(dir_save_train_np, 'label_%03d.npy' % i), label_)
#     np.save(os.path.join(dir_save_train_np, 'input_%03d.npy' % i), input_)

# ## 선택된 test 이미지를 npy 파일로 저장
# for i, (test_mask, test_original) in enumerate(zip(test_mask_list, test_original_list)):

#     label_ = np.asarray(test_mask)
#     input_ = np.asarray(test_original)

#     np.save(os.path.join(dir_save_test_np, 'label_%03d.npy' % i), label_)
#     np.save(os.path.join(dir_save_test_np, 'input_%03d.npy' % i), input_)

# ## 선택된 val 이미지를 npy 파일로 저장
# for i, (val_mask, val_original) in enumerate(zip(val_mask_list, val_original_list)):

#     label_ = np.asarray(val_mask)
#     input_ = np.asarray(val_original)

#     np.save(os.path.join(dir_save_val_np, 'label_%03d.npy' % i), label_)
#     np.save(os.path.join(dir_save_val_np, 'input_%03d.npy' % i), input_)


In [3]:
## 라이브러리 불러오기
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets
import torch.nn.functional as F

In [4]:

## 네트워크 구축하기
class ResUNet(nn.Module):
    def __init__(self):
        super(ResUNet, self).__init__()

        # Convolution + BatchNormalization + Relu 정의하기
        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True): 
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride, padding=padding,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        class ResidualBlock(nn.Module):
            def __init__(self, in_channels, out_channels):
                super(ResidualBlock, self).__init__()
                self.conv_block = nn.Sequential(
                    nn.BatchNorm2d(in_channels),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(out_channels),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                )
                """ Residual block과 channel size를 맞추기 위한 conv operation """ #kernel size는 동일해야함
                self.shortcut = nn.Sequential(
                    nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                ) 
            
            def forward(self, inputs):
                r = self.conv_block(inputs)
                s = self.shortcut(inputs)
                
                skip = r + s
                return skip

        # 수축 경로(Contracting path)
        self.enc1_1 = ResidualBlock(in_channels=1, out_channels=64)
        self.enc1_2 = ResidualBlock(in_channels=64, out_channels=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = ResidualBlock(in_channels=64, out_channels=128)
        self.enc2_2 = ResidualBlock(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = ResidualBlock(in_channels=128, out_channels=256)
        self.enc3_2 = ResidualBlock(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = ResidualBlock(in_channels=256, out_channels=512)
        self.enc4_2 = ResidualBlock(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = ResidualBlock(in_channels=512, out_channels=1024)
        
        # 확장 경로(Expansive path)
        self.dec5_1 = ResidualBlock(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,  # Adjust output channels to 512
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = ResidualBlock(in_channels=2 * 512, out_channels=512)  # Adjust input channels to 2 * 512
        self.dec4_1 = ResidualBlock(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = ResidualBlock(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = ResidualBlock(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = ResidualBlock(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = ResidualBlock(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = ResidualBlock(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = ResidualBlock(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)


    # forward 함수 정의하기
    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)

        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x


In [5]:
# 모델의 state_dict 출력
# print("Model's state_dict:")
# for param_tensor in net.state_dict():
#     print(param_tensor, "\t", net.state_dict()[param_tensor].size())

## 3. DataLoader 및 Transform 정의하기

#### 1) DataLoader  
네트워크에 데이터를 올리기 위해 DataLoader로 구현해주겠습니다.


In [6]:
# 데이터 로더를 구현하기
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_data = os.listdir(self.data_dir)

        lst_label = [f for f in lst_data if f.startswith('label')]
        lst_input = [f for f in lst_data if f.startswith('input')]

        lst_label.sort()
        lst_input.sort()

        self.lst_label = lst_label
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        # 정규화
        label = label/255.0
        input = input/255.0

        # 이미지와 레이블의 차원 = 2일 경우(채널이 없을 경우, 흑백 이미지), 새로운 채널(축) 생성
        if label.ndim == 2:
            label = label[:, :, np.newaxis]
        if input.ndim == 2:
            input = input[:, :, np.newaxis]

        data = {'input': input, 'label': label}

        # transform이 정의되어 있다면 transform을 거친 데이터를 불러옴
        if self.transform:
            data = self.transform(data)

        return data

#### 2) Transform

- ToTensor : numpy에서 tensor로 데이터 type을 변경
- Normalization : 데이터 정규화
- RandomFlip : 랜덤 좌우 및 상하 반전


In [8]:
# 트렌스폼 구현하기
class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = input.transpose((2, 0, 1)).astype(np.float32)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data

class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        label, input = data['label'], data['input']

        input = (input - self.mean) / self.std

        data = {'label': label, 'input': input}

        return data

class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            label = np.fliplr(label)
            input = np.fliplr(input)

        if np.random.rand() > 0.5:
            label = np.flipud(label)
            input = np.flipud(input)

        data = {'label': label, 'input': input}

        return data

- 정규화가 잘 수행된 것을 볼 수 있다.

## 4. 네트워크 저장/불러오는 함수 정의하기


In [9]:
## 네트워크 저장하기
def save(ckpt_dir, net, optim, epoch, best=False):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    if best:
        torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
                   "%s/model_best.pth" % ckpt_dir)
    # else:
    #     torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
    #                "%s/model_epoch%d.pth" % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst = [f for f in ckpt_lst if f.endswith(".pth")]
    ckpt_lst = sorted(ckpt_lst, key=lambda f: int(''.join(filter(str.isdigit, f.split(".")[0]))) if f.split(".")[0].isdigit() else -1)

    if not ckpt_lst:
        epoch = 0
        return net, optim, epoch

    dict_model = torch.load(os.path.join(ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    
    # 파일명에서 epoch 추출 시 예외 처리
    try:
        epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])
    except IndexError:
        epoch = 0

    return net, optim, epoch




## 5. 네트워크 훈련하기

In [10]:
def calculate_iou(prediction, ground_truth):
    prediction = prediction.cpu().numpy()  # CUDA 텐서를 CPU로 이동하여 NumPy 배열로 변환
    ground_truth = ground_truth.cpu().numpy()  # CUDA 텐서를 CPU로 이동하여 NumPy 배열로 변환
    
    intersection = np.logical_and(prediction, ground_truth)
    union = np.logical_or(prediction, ground_truth)
    iou = np.sum(intersection) / np.sum(union)
    return iou


In [11]:
# 훈련 파라미터 설정하기
lr = 1e-4 #0.0001
batch_size = 4
num_epoch = 100

base_dir = r'C:\Users\KDJ\Desktop\KDJ\Code'
data_dir = r'C:\Users\KDJ\Desktop\KDJ\Code\data\npy_data'
ckpt_dir = os.path.join(base_dir, "checkpoint")
log_dir = os.path.join(base_dir, "log")


# 훈련을 위한 Transform과 DataLoader
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])

dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=transform)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)

dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=0)

# 네트워크 생성하기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = ResUNet().to(device)

# 손실함수 정의하기
fn_loss = nn.BCEWithLogitsLoss().to(device)

# Optimizer 설정하기
#optim = torch.optim.ASGD(net.parameters(), lr=lr)
optim = torch.optim.Adam(net.parameters(), lr=lr)

# 그밖에 부수적인 variables 설정하기
num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

num_batch_train = np.ceil(num_data_train / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

# 그 밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

# Tensorboard 를 사용하기 위한 SummaryWriter 설정
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

# 네트워크 학습시키기
st_epoch = 0
# 학습한 모델이 있을 경우 모델 로드하기
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim) 

best_loss = float('inf')

# for epoch in range(st_epoch + 1, num_epoch + 1):
#         net.train()
#         loss_arr = []
#         iou_arr = []  # IoU 값을 저장하기 위한 리스트 추가

#         for batch, data in enumerate(loader_train, 1):
#             # forward pass
#             label = data['label'].to(device)
#             input = data['input'].to(device)

#             # print(np.shape(input))
#             # print(np.shape(label))

#             output = net(input)

#             # backward pass
#             optim.zero_grad()

#             loss = fn_loss(output, label)
#             loss.backward()

#             optim.step()

#             # 손실함수 계산
#             loss_arr += [loss.item()]

#             # IoU 계산
#             prediction = fn_class(output)
#             iou = calculate_iou(prediction, label)
#             iou_arr += [iou]

#             print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f | IOU %.4f" %
#                     (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr), np.mean(iou_arr)))
            
#             # Tensorboard 저장하기
#             label = fn_tonumpy(label)
#             input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
#             output = fn_tonumpy(fn_class(output))

#             writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
#             writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
#             writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

#         writer_train.add_scalar('loss', np.mean(loss_arr), epoch)
#         writer_train.add_scalar('iou', np.mean(iou_arr), epoch)

#         with torch.no_grad():
#             net.eval()
#             loss_arr = []
#             iou_arr = []  # IoU 값을 저장하기 위한 리스트 추가


#             for batch, data in enumerate(loader_val, 1):
#                 # forward pass
#                 label = data['label'].to(device)
#                 input = data['input'].to(device)

#                 output = net(input)

#                 # 손실함수 계산하기
#                 loss = fn_loss(output, label)

#                 loss_arr += [loss.item()]

#                 # IoU 계산
#                 prediction = fn_class(output)
#                 iou = calculate_iou(prediction, label)
#                 iou_arr += [iou]

#                 print("VAILD: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f | IOU %.4f" %
#                     (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr), np.mean(iou_arr)))
            
#                 # Tensorboard 저장하기
#                 label = fn_tonumpy(label)
#                 input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
#                 output = fn_tonumpy(fn_class(output))

#                 writer_val.add_image('label', label, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
#                 writer_val.add_image('input', input, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
#                 writer_val.add_image('output', output, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')

#         writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
#         writer_val.add_scalar('iou', np.mean(iou_arr), epoch)

#         mean_loss = np.mean(loss_arr)
#         # mean_iou = calculate_mean_iou(iou_arr, label, num_classes)  # 수정된 부분


#          # Best model 만 저장하기
#         if mean_loss < best_loss:
#             best_loss = mean_loss
#             save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch, best=True)
#         else:
#             save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch, best=False)

#         writer_train.close()
#         writer_val.close()

# #텐서보드로 그래프 보기
# #tensorboard --logdir ./log 

## 6. 훈련된 모델 테스트하기

In [14]:
transform = transforms.Compose([Normalization(mean=0.5, std=0.5), ToTensor()])

dataset_test = Dataset(data_dir=os.path.join(data_dir, 'test'), transform=transform)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=0)

# 그밖에 부수적인 variables 설정하기
num_data_test = len(dataset_test)
num_batch_test = np.ceil(num_data_test / batch_size)

# 결과 디렉토리 생성하기
result_dir = os.path.join(base_dir, 'result')
if not os.path.exists(result_dir):
    os.makedirs(os.path.join(result_dir, 'png'))
    os.makedirs(os.path.join(result_dir, 'numpy'))


net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

with torch.no_grad():
      net.eval()
      loss_arr = []

      for batch, data in enumerate(loader_test, 1):
          # forward pass
          label = data['label'].to(device)
          input = data['input'].to(device)

          output = net(input)

          # 손실함수 계산하기
          loss = fn_loss(output, label)

          loss_arr += [loss.item()]

          print("TEST: BATCH %04d / %04d | LOSS %.4f" %
                (batch, num_batch_test, np.mean(loss_arr)))

          # Tensorboard 저장하기
          label = fn_tonumpy(label)
          input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
          output = fn_tonumpy(fn_class(output))

          # 테스트 결과 저장하기
          for j in range(label.shape[0]):
              id = num_batch_test * (batch - 1) + j

              plt.imsave(os.path.join(result_dir, 'png', 'label_%04d.png' % id), label[j].squeeze(), cmap='gray')
              plt.imsave(os.path.join(result_dir, 'png', 'input_%04d.png' % id), input[j].squeeze(), cmap='gray')
              plt.imsave(os.path.join(result_dir, 'png', 'output_%04d.png' % id), output[j].squeeze(), cmap='gray')

              np.save(os.path.join(result_dir, 'numpy', 'label_%04d.npy' % id), label[j].squeeze())
              np.save(os.path.join(result_dir, 'numpy', 'input_%04d.npy' % id), input[j].squeeze())
              np.save(os.path.join(result_dir, 'numpy', 'output_%04d.npy' % id), output[j].squeeze())

print("AVERAGE TEST: BATCH %04d / %04d | LOSS %.4f" %
        (batch, num_batch_test, np.mean(loss_arr)))

TEST: BATCH 0001 / 0040 | LOSS 0.0723
TEST: BATCH 0002 / 0040 | LOSS 0.0671
TEST: BATCH 0003 / 0040 | LOSS 0.0758
TEST: BATCH 0004 / 0040 | LOSS 0.0786
TEST: BATCH 0005 / 0040 | LOSS 0.0795
TEST: BATCH 0006 / 0040 | LOSS 0.0785
TEST: BATCH 0007 / 0040 | LOSS 0.0786
TEST: BATCH 0008 / 0040 | LOSS 0.0772
TEST: BATCH 0009 / 0040 | LOSS 0.0758
TEST: BATCH 0010 / 0040 | LOSS 0.0754
TEST: BATCH 0011 / 0040 | LOSS 0.0779
TEST: BATCH 0012 / 0040 | LOSS 0.0779
TEST: BATCH 0013 / 0040 | LOSS 0.0783
TEST: BATCH 0014 / 0040 | LOSS 0.0792
TEST: BATCH 0015 / 0040 | LOSS 0.0786
TEST: BATCH 0016 / 0040 | LOSS 0.0790
TEST: BATCH 0017 / 0040 | LOSS 0.0802
TEST: BATCH 0018 / 0040 | LOSS 0.0796
TEST: BATCH 0019 / 0040 | LOSS 0.0797
TEST: BATCH 0020 / 0040 | LOSS 0.0817
TEST: BATCH 0021 / 0040 | LOSS 0.0821
TEST: BATCH 0022 / 0040 | LOSS 0.0809
TEST: BATCH 0023 / 0040 | LOSS 0.0802
TEST: BATCH 0024 / 0040 | LOSS 0.0818
TEST: BATCH 0025 / 0040 | LOSS 0.0812
TEST: BATCH 0026 / 0040 | LOSS 0.0818
TEST: BATCH 

## 7. 결과 시각화하기

In [ ]:
for i in range(10):
    ##
    lst_data = os.listdir(os.path.join(result_dir, 'numpy'))

    lst_label = [f for f in lst_data if f.startswith('label')]
    lst_input = [f for f in lst_data if f.startswith('input')]
    lst_output = [f for f in lst_data if f.startswith('output')]

    lst_label.sort()
    lst_input.sort()
    lst_output.sort()

    ##
    id = i

    label = np.load(os.path.join(result_dir,"numpy", lst_label[id]))
    input = np.load(os.path.join(result_dir,"numpy", lst_input[id]))
    output = np.load(os.path.join(result_dir,"numpy", lst_output[id]))

    ## 플롯 그리기
    plt.figure(figsize=(8,6))
    plt.subplot(131)
    plt.imshow(input, cmap='gray')
    plt.title('input')

    plt.subplot(132)
    plt.imshow(label, cmap='gray')
    plt.title('label')

    plt.subplot(133)
    plt.imshow(output, cmap='gray')
    plt.title('output')

    plt.show()


In [12]:
#npy 파일로 전처리

infer_data_paths = glob.glob('/Users/KDJ/Desktop/thyroid2/variable/*png')

dir_save_infer_np = '/Users/KDJ/Desktop/thyroid2/infer_np'

if not os.path.exists(dir_save_infer_np):
    os.makedirs(dir_save_infer_np)

for i, infer_data_path in enumerate(infer_data_paths):
    infer_data = np.array(Image.open(infer_data_path).resize((256, 256)).convert('L'))
    input_ = np.asarray(infer_data)
    np.save(os.path.join(dir_save_infer_np, 'input_%03d.npy' % i), input_)


In [13]:
# 데이터 로더를 구현하기
class Dataset_infer(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        lst_data = os.listdir(self.data_dir)

        lst_input = [f for f in lst_data if f.startswith('input')]

        lst_input.sort()

        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_input)

    def __getitem__(self, index):
        input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        # 정규화
        input = input/255.0

        # 이미지와 레이블의 차원 = 2일 경우(채널이 없을 경우, 흑백 이미지), 새로운 채널(축) 생성
        if input.ndim == 2:
            input = input[:, :, np.newaxis]

        data = {'input': input}

        # transform이 정의되어 있다면 transform을 거친 데이터를 불러옴
        if self.transform:
            data = self.transform(data)

        return data

In [14]:
# 트렌스폼 구현하기
class ToTensor_infer(object):
    def __call__(self, data):
        input = data['input']

        input = input.transpose((2, 0, 1)).astype(np.float32)

        data = {'input': torch.from_numpy(input)}

        return data

class Normalization_infer:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        input = data['input']
        input = (input - self.mean) / self.std
        data = {'input': input}
        return data

In [15]:
base_dir = r'C:\Users\KDJ\Desktop\KDJ\Code'
infer_base_dir = r'C:\Users\KDJ\Desktop\thyroid2'
infer_data_dir = r"C:\Users\KDJ\Desktop\thyroid2"

ckpt_dir = os.path.join(base_dir, "checkpoint")

transform = transforms.Compose([Normalization_infer(mean=0.5, std=0.5), ToTensor_infer()])

dataset_test = Dataset_infer(data_dir=os.path.join(infer_data_dir, 'infer_np'), transform=transform)
loader_test = DataLoader(dataset_test, batch_size=8, shuffle=False, num_workers=0)

num_data_test = len(dataset_test)
num_batch_test = np.ceil(num_data_test / batch_size)

# 결과 디렉토리 생성하기
result_dir = os.path.join(infer_base_dir, 'result')
if not os.path.exists(result_dir):
    os.makedirs(os.path.join(result_dir, 'png'))
    os.makedirs(os.path.join(result_dir, 'numpy'))

net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

with torch.no_grad():
    net.eval()

    for batch, data in enumerate(loader_test, 1):
        # forward pass
        input = data['input'].to(device)

        output = net(input)

        # Tensorboard 저장하기
        input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_class(output))

        # 테스트 결과 저장하기
        for j in range(input.shape[0]):
            id = num_batch_test * (batch - 1) + j

            plt.imsave(os.path.join(result_dir, 'png', 'input_%04d.png' % id), input[j].squeeze(), cmap='gray')
            plt.imsave(os.path.join(result_dir, 'png', 'output_%04d.png' % id), output[j].squeeze(), cmap='gray')

            np.save(os.path.join(result_dir, 'numpy', 'input_%04d.npy' % id), input[j].squeeze())
            np.save(os.path.join(result_dir, 'numpy', 'output_%04d.npy' % id), output[j].squeeze())


In [ ]:
for i in range(10):
    ##
    lst_data = os.listdir(os.path.join(result_dir, 'numpy'))

    lst_input = [f for f in lst_data if f.startswith('input')]
    lst_output = [f for f in lst_data if f.startswith('output')]

    lst_input.sort()
    lst_output.sort()

    ##
    id = i

    input = np.load(os.path.join(result_dir,"numpy", lst_input[id]))
    output = np.load(os.path.join(result_dir,"numpy", lst_output[id]))

    ## 플롯 그리기
    plt.figure(figsize=(8,6))
    plt.subplot(131)
    plt.imshow(input, cmap='gray')
    plt.title('input')

    plt.subplot(132)
    plt.imshow(output, cmap='gray')
    plt.title('output')

    plt.show()


In [21]:
ResUnet

NameError: name 'ResUnet' is not defined